In [1]:
from matplotlib import pylab
import nengo
import random
import numpy as np
import gzip as gz
import cPickle
from cPickle import load
try:
    import Image
except ImportError:
    from PIL import Image
from scipy.sparse.linalg import svds
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#%matplotlib inline #Makes visualizations appar inline (Commented out because animation popup as new window)

In [2]:
def rotate_img(img, degrees):
    '''Rotates image the degrees passed in counterclockwise
    Reshapes image to original shape
    '''
    original = img.shape
    
    newImg = scipy.ndimage.interpolation.rotate(np.reshape(img, (dim,dim), 'F'),degrees,reshape=False)
    newImg = np.reshape(newImg, original, 'F')
    return newImg

In [71]:
conn_synapse = 0.1 #post synaptic time constant to use for filtering (pstc) - what does changing this do?
probe_synapse = 0.01 #pstc
#multiplier = 2 #not used
n_neurons = 5000
direct = False #Direct - function computed explicitly instead of in neurons 
stop_time = 3.0
run_time = 3.0 #in seconds

In [34]:
dim = 7 #size of the image

img = np.zeros((dim,dim)) #create a simple image
img[3] = 1 #image is a horizontal line

img = img.flatten()

#View the image
#print(img)
#pylab.imshow(np.reshape(img, (dim,dim), 'F').T, cmap='Greys_r')
#plt.show()

In [35]:
def stim_func(t):
    '''returns the image for first 0.1s'''
    if t < 0.1:
        return img
    else:
        return [0 for _ in range(len(img))]

In [76]:
def connection_func(x):
    '''takes the output from the first ensemble and rotates it 10 degree'''
    return rotate_img(x,10)

In [77]:
with nengo.Network() as net:
    
    if direct:
        neuron_type = nengo.Direct() #function computed explicitly, instead of in neurons
    else:
        neuron_type = nengo.LIF() #spiking version of the leaky integrate-and-fire neuron model

    #Input stimulus - provide data to the ensemble
    ipt = nengo.Node(stim_func)
    
    #Group of neurons that collectively represent information(vector)
    ens = nengo.Ensemble(n_neurons,
                         dimensions=dim**2, #pixels of the image 
                         neuron_type=neuron_type)

    nengo.Connection(ipt, #source nengo object
                     ens, #destination object
                     synapse=None, #pstc
                     transform=1) #linear transformation, what does changing this do?
    
    conn = nengo.Connection(ens, ens, synapse=conn_synapse,transform=1, function=connection_func)
    
    probe = nengo.Probe(ens, attr='decoded_output',#sample_every=0.001,
                       synapse=probe_synapse)
   

In [78]:
sim = nengo.Simulator(net)

In [79]:
sim.run(run_time)

Simulation finished in 0:00:04.                                                 


In [13]:
pylab.imshow(np.reshape(img, (dim,dim), 'F'), cmap='Greys_r')

In [14]:
'''Image at stop time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][int(stop_time*1000)-1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [15]:
'''Image at start time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [82]:
'''Animation for Probe output'''
fig = plt.figure()

def updatefig(i):
    im = pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][i]],
                                 (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)
    
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=1, blit=True)
plt.show()


In [33]:
# save the output
#cPickle.dump(sim.data[probe], open( "Buffer_rotations_connection_direct.p", "wb" ) )
#cPickle.dump(sim.data[probe], open( "Buffer_rotations_connection_LIF.p", "wb" ) )
#cPickle.dump(sim.data[probe], open( "Buffer_rotations_connection_LIF_4000.p", "wb" ) )